In [ ]:
import pandas as pd
from open_geodata import geo

from sp_mpsp_divadmin.paths import input_path_tab, output_path_tabs

In [ ]:
df_mun = geo.load_dataset('tab.sp.tab_municipio_nome')

df_mun.info()
df_mun.head()

In [ ]:
# Dataframe
df_ids = pd.read_csv(
    input_path_tab / 'tab_ids_municipio.csv',
    # usecols=['id_municipio', 'id_gaema'],
    usecols=['id_municipio', 'id_gaema', 'id_rp'],
)

# Results
df_ids.info()
df_ids.head()

In [ ]:
# Merge Municípios
df = pd.merge(
    df_mun,
    df_ids,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio',
)

# Results
df.info()
df.head()

<br>

## GAEMA


In [ ]:
mask = (~df['id_gaema'].isnull()) & (df['id_rp'].isnull())
df[mask]

In [ ]:
# Dataframe GAEMAs
df_gaema = pd.read_csv(
    input_path_tab / 'tab_gaema.csv',
    usecols=['id', 'nome'],
)
df_gaema = df_gaema.add_prefix('gaema_')

# df_gaema['gaema_nome'] = df_gaema['gaema_nome'].add_prefix('')
df_gaema['gaema_nome'].replace('Núcleo', '', regex=True, inplace=True)
df_gaema['gaema_nome'] = df_gaema['gaema_nome'].apply(lambda x: f'GAEMA - {x}')
df_gaema['unidade'] = df_gaema['gaema_nome']

# Results
df_gaema.info()
df_gaema.head()

In [ ]:
# Merge GAEMAs
df_gaema_merged = pd.merge(
    df[mask],
    df_gaema,
    how='left',
    left_on='id_gaema',
    right_on='gaema_id',
)


df_gaema_merged.info()
df_gaema_merged.head()

<br>

## Rede Protetiva


In [ ]:
mask = (df['id_gaema'].isnull()) & (~df['id_rp'].isnull())
df[mask]

In [ ]:
# Dataframe Rede Protetiva
df_rp = pd.read_csv(
    input_path_tab / 'tab_redeprotetiva.csv',
    usecols=['id', 'nome'],
)
df_rp = df_rp.add_prefix('rp_')

# df_gaema['gaema_nome'] = df_gaema['gaema_nome'].add_prefix('')
# df_rp['rp_nome'].replace('Núcleo', '', regex=True, inplace=True)
df_rp['rp_nome'] = df_rp['rp_nome'].apply(lambda x: f'Rede Protetiva - {x}')
df_rp['unidade'] = df_rp['rp_nome']

# Results
df_rp.info()
df_rp.head()

In [ ]:
# Merge GAEMAs
df_rp_merged = pd.merge(
    df[mask],
    df_rp,
    how='left',
    left_on='id_rp',
    right_on='rp_id',
)

df_rp_merged.info()
df_rp_merged.head()

In [ ]:
df_final = pd.concat([df_gaema_merged, df_rp_merged])
df_final.drop(
    ['id_gaema', 'id_rp', 'gaema_id', 'gaema_nome', 'rp_id', 'rp_nome'],
    axis=1,
    inplace=True,
    errors='ignore',
)

df_final.info()
df_final.head()

In [ ]:
df_final['id_municipio_6d'] = df_final['id_municipio'].astype(str).str[:6]
df_final

In [ ]:
df_final.to_csv(
    output_path_tabs / 'tab_gaemas_rps.csv',
    index=False,
)